In [ ]:
# Regressão linear e DNN com TensorFlow
# Vamos o valor do aluguem em Alagoas

In [1]:
!pip install -q seaborn

In [65]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# Vamos fazer o numpy printouts mais fácil de ler.

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
import tensorflow as tf

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
print(tf.__version__)

2.5.0


In [ ]:
# Vamos baixar a base de dados

In [5]:
raw_dataset=pd.DataFrame(pd.read_csv('C:/Users/Cássio/Desktop/AP/AP/censo2010.csv',engine='python'))
dataset = raw_dataset.copy()
dataset.tail()

,Unnamed: 0,UF,Codigo_do_Municipio,Area_de_Ponderacao,Controle,Peso_Amostral,Regiao,Codigo_da_Mesorregiao,Codigo_da_Microrregiao,Codigo_da_Regiao_Metropolitana,...,Marca_de_imputacao_na_v0218,Marca_de_imputacao_na_v0219,Marca_de_imputacao_na_v0220,Marca_de_imputacao_na_v0221,Marca_de_imputacao_na_v02022,Marca_de_imputacao_na_v0301,Marca_de_imputacao_na_v0401,Marca_de_imputacao_na_v0402,Marca_de_imputacao_na_v0701,Situacao_do_setor
78339,78340,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78340,78341,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78341,78342,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78342,78343,16,808,1600808001001,6186781,4,4,31,106,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78343,78344,16,808,1600808001001,6186781,4,4,31,106,75,...,0.0,1.0,5.0,9.0,9.0,9.0,9.0,9.0,NaN,NaN


In [ ]:
# como a base de dados é muito grande para trabalhar em um computador pessoal, vamos pegar só 1% da mesma
# se for para utilizar processamento distribuido dá pra fazer tranquilamente com a base toda

In [6]:
n=1
dataset=dataset.head(int(len(dataset)*(n/100)))

In [7]:
dataset.tail()

,Unnamed: 0,UF,Codigo_do_Municipio,Area_de_Ponderacao,Controle,Peso_Amostral,Regiao,Codigo_da_Mesorregiao,Codigo_da_Microrregiao,Codigo_da_Regiao_Metropolitana,...,Marca_de_imputacao_na_v0218,Marca_de_imputacao_na_v0219,Marca_de_imputacao_na_v0220,Marca_de_imputacao_na_v0221,Marca_de_imputacao_na_v02022,Marca_de_imputacao_na_v0301,Marca_de_imputacao_na_v0401,Marca_de_imputacao_na_v0402,Marca_de_imputacao_na_v0701,Situacao_do_setor
778,779,16,55,1600055001001,3522415,3,0,87,219,65,...,0.0,1.0,3.0,0.0,3.0,5.0,3.0,6.0,NaN,NaN
779,780,16,55,1600055001001,3574002,2,8,39,90,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,781,16,55,1600055001001,3574002,2,8,39,90,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,782,16,55,1600055001001,3574002,2,8,39,90,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,783,16,55,1600055001001,3580775,2,1,10,446,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# vamos ver um sumário dos dados

In [8]:
dataset.describe()

,Unnamed: 0,UF,Codigo_do_Municipio,Area_de_Ponderacao,Controle,Peso_Amostral,Regiao,Codigo_da_Mesorregiao,Codigo_da_Microrregiao,Codigo_da_Regiao_Metropolitana,...,Marca_de_imputacao_na_v0218,Marca_de_imputacao_na_v0219,Marca_de_imputacao_na_v0220,Marca_de_imputacao_na_v0221,Marca_de_imputacao_na_v02022,Marca_de_imputacao_na_v0301,Marca_de_imputacao_na_v0401,Marca_de_imputacao_na_v0402,Marca_de_imputacao_na_v0701,Situacao_do_setor
count,783.000000,783.0,783.0,7.830000e+02,7.830000e+02,783.000000,783.000000,783.000000,783.000000,783.000000,...,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,0.0,0.0
mean,392.000000,16.0,55.0,1.600055e+12,1.702056e+06,2.615581,4.688378,50.664112,521.650064,51.265645,...,0.272109,1.360544,5.653061,0.394558,1.108844,4.081633,1.585034,3.360544,NaN,NaN
std,226.176922,0.0,0.0,0.000000e+00,9.668430e+05,1.057968,2.923134,29.255744,291.319035,28.285102,...,1.455071,1.384580,1.180046,1.773310,2.407549,2.631501,2.531564,2.556071,NaN,NaN
min,1.000000,16.0,55.0,1.600055e+12,1.266600e+04,1.000000,0.000000,0.000000,7.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,196.500000,16.0,55.0,1.600055e+12,8.941770e+05,2.000000,2.000000,25.000000,268.000000,26.000000,...,0.000000,1.000000,6.000000,0.000000,0.000000,3.000000,0.000000,0.500000,NaN,NaN
50%,392.000000,16.0,55.0,1.600055e+12,1.724893e+06,3.000000,5.000000,50.000000,547.000000,53.000000,...,0.000000,1.000000,6.000000,0.000000,0.000000,5.000000,0.000000,3.000000,NaN,NaN
75%,587.500000,16.0,55.0,1.600055e+12,2.513747e+06,3.000000,7.000000,77.000000,772.000000,77.000000,...,0.000000,1.000000,6.000000,0.000000,0.000000,6.000000,3.000000,5.000000,NaN,NaN
max,783.000000,16.0,55.0,1.600055e+12,3.580775e+06,6.000000,9.000000,99.000000,996.000000,99.000000,...,8.000000,8.000000,8.000000,9.000000,9.000000,9.000000,9.000000,9.000000,NaN,NaN


In [ ]:
# A nossa base de dados contem valores faltantes, vamo ver quantos valores faltantes há em cada linha

In [9]:
dataset.isnull().sum()

Unnamed: 0                                                0
UF                                                        0
Codigo_do_Municipio                                       0
Area_de_Ponderacao                                        0
Controle                                                  0
Peso_Amostral                                             0
Regiao                                                    0
Codigo_da_Mesorregiao                                     0
Codigo_da_Microrregiao                                    0
Codigo_da_Regiao_Metropolitana                            0
Situacao_do_Domicilio                                     0
Especie_de_Unidade_Visitada                               0
Tipo_de_Especie                                           0
Domicilio_condicao_de_ocupacao                            0
Valor_do_aluguel_(em_reais)                               0
Aluguel_em_nº_de_salarios_minimos                         0
Material_predominante_nas_paredes_extern

In [ ]:
# Como eu já verifiquei as variáveis que quero, vou selecionar na base apenas estas variáveis

In [10]:
dataset=dataset[['Regiao','Valor_do_aluguel_(em_reais)','Nº_de_comodos','Comodos_como_dormitorio_numero']]

In [11]:
dataset=pd.get_dummies(dataset, columns=['Regiao'])

In [12]:
dataset=dataset.rename(columns={'Valor_do_aluguel_(em_reais)':'Valor_do_aluguel_em_reais'})

In [13]:
dataset=dataset.rename(columns={'Nº_de_comodos':'N_de_comodos'})

In [ ]:
# Para facilitar vamos deletar as linhas com valores nulos para deixar este tutorial inicial simples

In [14]:
dataset = dataset.dropna()

In [15]:
dataset

,Valor_do_aluguel_em_reais,N_de_comodos,Comodos_como_dormitorio_numero,Regiao_0,Regiao_1,Regiao_2,Regiao_3,Regiao_4,Regiao_5,Regiao_6,Regiao_7,Regiao_8,Regiao_9
0,20030,12,0,0,0,0,0,0,0,0,0,1,0
1,20030,42,20,0,0,0,0,0,0,0,0,1,0
2,20030,32,50,0,0,0,0,0,0,0,0,1,0
3,20030,21,30,0,0,0,0,0,0,0,0,1,0
4,20030,51,90,0,0,0,0,0,0,0,1,0,0
5,20030,11,70,0,0,0,0,0,0,0,1,0,0
6,20030,22,80,0,0,0,0,0,0,0,1,0,0
7,20030,61,40,0,0,0,0,0,0,0,1,0,0
8,20030,72,20,0,0,0,0,0,0,0,1,0,0
9,20030,31,20,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Vamos separar a base de dados entre treino e teste

In [16]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
# para a etapa de inspecionar a base, vamos transformar os valores float em int (ATENÇÃO, ACHO ESTE PASSO DESNECESSÁRIO)

In [17]:
dataset.astype(int)

,Valor_do_aluguel_em_reais,N_de_comodos,Comodos_como_dormitorio_numero,Regiao_0,Regiao_1,Regiao_2,Regiao_3,Regiao_4,Regiao_5,Regiao_6,Regiao_7,Regiao_8,Regiao_9
0,20030,12,0,0,0,0,0,0,0,0,0,1,0
1,20030,42,20,0,0,0,0,0,0,0,0,1,0
2,20030,32,50,0,0,0,0,0,0,0,0,1,0
3,20030,21,30,0,0,0,0,0,0,0,0,1,0
4,20030,51,90,0,0,0,0,0,0,0,1,0,0
5,20030,11,70,0,0,0,0,0,0,0,1,0,0
6,20030,22,80,0,0,0,0,0,0,0,1,0,0
7,20030,61,40,0,0,0,0,0,0,0,1,0,0
8,20030,72,20,0,0,0,0,0,0,0,1,0,0
9,20030,31,20,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Vamos ver uma estatística geral dos dados

In [18]:
train_dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Valor_do_aluguel_em_reais,626.0,20030.000000,0.000000,20030.0,20030.0,20030.0,20030.0,20030.0
N_de_comodos,626.0,31.025559,18.654569,1.0,12.0,31.0,41.0,92.0
Comodos_como_dormitorio_numero,626.0,43.897764,28.048229,0.0,20.0,40.0,70.0,90.0
Regiao_0,626.0,0.108626,0.311418,0.0,0.0,0.0,0.0,1.0
Regiao_1,626.0,0.095847,0.294616,0.0,0.0,0.0,0.0,1.0
Regiao_2,626.0,0.059105,0.236011,0.0,0.0,0.0,0.0,1.0
Regiao_3,626.0,0.094249,0.292409,0.0,0.0,0.0,0.0,1.0
Regiao_4,626.0,0.113419,0.317357,0.0,0.0,0.0,0.0,1.0
Regiao_5,626.0,0.067093,0.250382,0.0,0.0,0.0,0.0,1.0
Regiao_6,626.0,0.102236,0.303201,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Vamos separar a variável que queremos prever das demais
# primeiro vamos copiar nossa base de dados em train_features (cópia da base treino) e test_features (cópia da base teste)
# e em seguida retiramos as variáveis independentes desta base, deixando nela apenas as variáveis independentes (feature) e criando a
# novas bases de teste e treino apenas com a dependente (respectivamente test_labels e train_labels)

In [ ]:
# OBS: SE EU QUISER APLICAR UMA REGRESSÃO SIMPLES, BASTA COLOCAR APENAS UMA VARIÁVEL NA BASE FEATURES

In [19]:
train_labels = train_dataset.copy()
test_labels = test_dataset.copy()

In [20]:
train_features=pd.concat([train_labels.pop(x) for x in ['N_de_comodos','Comodos_como_dormitorio_numero','Regiao_0','Regiao_1','Regiao_2','Regiao_3','Regiao_4','Regiao_5','Regiao_6','Regiao_7','Regiao_8','Regiao_9']], axis=1)
test_features=pd.concat([test_labels.pop(x) for x in ['N_de_comodos','Comodos_como_dormitorio_numero','Regiao_0','Regiao_1','Regiao_2','Regiao_3','Regiao_4','Regiao_5','Regiao_6','Regiao_7','Regiao_8','Regiao_9']], axis=1)

In [ ]:
# Agora temos que normalizar os dados, ou seja, enquadrar as variáveis numéricas
# em um valor entre 0 e 1

In [21]:
train_dataset.describe().transpose()[['mean', 'std']]
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

[31.026 43.898  0.109  0.096  0.059  0.094  0.113  0.067  0.102  0.134
  0.125  0.101]


In [ ]:
# Vamos ver como ficaram os dados após a normalização

In [22]:
first = np.array(train_features[:1])

In [23]:
with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[21 70  0  0  0  0  0  0  1  0  0  0]]

Normalized: [[-0.54  0.93 -0.35 -0.33 -0.25 -0.32 -0.36 -0.27  2.96 -0.39 -0.38 -0.33]]


In [ ]:
# Preparados os dados, vamos aplicar a regressão linear
# múltipla com as variáveis independentes que estão na nossa base

In [24]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
# Vamos ver se o resultado deste modelo (abaixo apenas as 10 primeiras observações)

In [25]:
linear_model.predict(train_features[:10])

array([[ 1.246],
       [-1.947],
       [ 0.355],
       [ 2.221],
       [ 1.039],
       [ 0.381],
       [-1.677],
       [ 1.903],
       [-1.848],
       [-2.409]], dtype=float32)

In [ ]:
# Quando chamamos o modelo, sua matriz de pesos será construída (são os parâmetros de cada uma das variáveis independentes)

In [26]:
linear_model.layers[1].kernel

<tf.Variable 'dense/kernel:0' shape=(12, 1) dtype=float32, numpy=
array([[ 0.107],
       [ 0.407],
       [-0.479],
       [-0.34 ],
       [-0.64 ],
       [ 0.605],
       [ 0.54 ],
       [ 0.166],
       [ 0.316],
       [-0.59 ],
       [ 0.26 ],
       [ 0.451]], dtype=float32)>

In [ ]:
# Vamos utilizar novamente compile e fit para determinar os parâmetros que
# precisamos e logo em seguida treinar o modelo

In [27]:
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [28]:
%%time
history = linear_model.fit(
    train_features, train_labels, 
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

Wall time: 3.98 s


In [ ]:
# Agora vamos ver como variou o loss a medida que o modelo com uma regressão
# múltipla foi treinado

In [29]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
# Vamos guardar os resultados com base na base de teste para depois

In [30]:
test_results = {}

In [31]:
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)

In [ ]:
# Vamos agora aplicar uma regressão DNN
# Utilizaremos para tanto, vamos utilizar o método "compile" para treinar o modelo

In [32]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
# Utilizaremos todos os dados como insumo, melhoramos a
# a performance ao utilizarmos o modelo na base de dados de validação

In [33]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 12)                25        
_________________________________________________________________
dense_1 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 5,082
Trainable params: 5,057
Non-trainable params: 25
_________________________________________________________________


In [34]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

plot_loss(history)

Wall time: 4.17 s


In [ ]:
# Novamente vamos salvar os resultados da previsão em nossa base de teste

In [35]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

In [ ]:
# Agora que já fiz todas as estimações, vamos comparar os resultados dos modelos

In [36]:
pd.DataFrame(test_results, index=['Mean absolute error [Valor_do_Aluguel]']).T

,Mean absolute error [Valor_do_Aluguel]
dnn_model,36.827866
linear_model,19867.978516


In [ ]:
# É SEMPRE BOM LEMBRAR QUE MATRIZ DE CONFUSÃO E ACCURACY SCORE NÃO FAZEM SENTIDO PARA AVALIAR
# UM MODELO DE REGRESSÃO, APENAS DE CLASSIFICAÇÃO. EM QUALQUER MODELO DE REGRESSÃO, POR MELHOR
# QUE SEJA, A CHANCE DE ACERTAR EXATAMENTE O VALOR ATÉ NAS CASAS DECIMAIS MÍNIMA

In [ ]:
# Finally, predict have a look at the errors made by the model when making predictions on the test set:

In [37]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Valor_do_Aluguel]')
plt.ylabel('Predictions [Valor_do_Aluguel]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
# Lembrando que abaixo iremos calcular o erro e, por isso, na função "error" abaixo transformamos test_labels em formato array
# pois ele estava em formato dataframe e para aplicar subtração ele deve ter o mesmo formato (array) do test_predictions, como
# vemos em seguida

In [ ]:
# Vamos dar uma olhada na distribuição dos erros:

In [40]:
error = test_predictions - np.array(test_labels)
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [Valor_do_aluguel]')
_ = plt.ylabel('Count')

In [44]:
dnn_model.save('dnn_model')

INFO:tensorflow:Assets written to: dnn_model\assets


In [ ]:
# E se você recarregar o modelo salvo, veja que este te dará o mesmo resultado
# de previsão, provando que o modelo foi salvo corretamente.

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)

pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T